# Inference PointCloud MMDetection3D

In [1]:
import pickle
import os

import open3d as o3d
import numpy as np
from open3d.web_visualizer import draw
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import mmdet3d

from mmdet3d.apis import inference_detector, init_model
# from sensus.utils.data_converter import pc2pc_object
from mmdet3d.utils import register_all_modules




Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [22]:
def generate_bbox(bbox_tensor, color):
    # Extract label info
    dimensions = bbox_tensor[3:6]
    position = bbox_tensor[0:3]
    rotation = bbox_tensor[6]

    print(position)
    
    # Generate bbox
    box = o3d.geometry.TriangleMesh.create_box(width=dimensions[0], height=dimensions[1], depth=dimensions[2])

    # Width is x, height is -y and z is up
    box.paint_uniform_color([1.0, 0.0, 0.0]) # Set color to red

    # Translate box
    box.translate(position)

    # Translate box to center
    box.translate([-dimensions[0]/2, -dimensions[1]/2, 0])

    # Rotate box
    center = box.get_center()
    # rotation = box.get_rotation_matrix_from_xyz((0, 0, -rotation - np.pi/2)) # Notice the negative sign for rotation
    rotation = box.get_rotation_matrix_from_xyz((0, 0, -rotation)) # Notice the negative sign for rotation

    box.rotate(rotation, center=center)

    lines = o3d.geometry.LineSet.create_from_triangle_mesh(box)
    # Remove lines that connect non-adjacent points
    lines.lines = o3d.utility.Vector2iVector(np.array([[0, 1], [2, 0], [2, 3], [3, 1], [4, 5], [4, 6], [6, 7], [7, 5], [0, 4], [1, 5], [2, 6], [3, 7]]))
    lines.paint_uniform_color(color)

    return lines

In [23]:
def generate_cars(num_cars, bbox_mmdet, bbox_dair):
    # Generate bbox
    boxes_mmdet = []
    boxes_dair = []
    for i in range(num_cars):
        boxes_mmdet.append(generate_bbox(bbox_mmdet[i], [1.0, 0.0, 0.0]))
        boxes_dair.append(generate_bbox(bbox_dair[i], [0.0, 1.0, 0.0]))

    return boxes_mmdet, boxes_dair    

In [24]:
register_all_modules()

## Model provided by mmdetection3d
config_mmdet = '../../mmdetection3d/configs/pointpillars/pointpillars_hv_secfpn_8xb6-160e_kitti-3d-car.py'
checkpoint_mmdet = '../../mmdetection3d/checkpoints/hv_pointpillars_secfpn_6x8_160e_kitti-3d-car_20220331_134606-d42d15ed.pth'

## Model trained on DAIR
config_dair = '../configs/dair_pointpillars/pointpillars_hv_secfpn_8xb6-160e_kitti-3d-car.py'
checkpoint_dair = '../../work_dirs/pointpillars_hv_secfpn_8xb6-160e_kitti-3d-car/epoch_80.pth'

## PCD file
pcd_path = 'DAIR/000000/000000.bin'

model_mmdet = init_model(config_mmdet, checkpoint_mmdet, device='cuda:1')
result_mmdet, data_mmdet = inference_detector(model_mmdet, pcd_path)

model_dair = init_model(config_dair, checkpoint_dair, device='cuda:1')
result_dair, data_dair = inference_detector(model_dair, pcd_path)


Loads checkpoint by local backend from path: ../../mmdetection3d/checkpoints/hv_pointpillars_secfpn_6x8_160e_kitti-3d-car_20220331_134606-d42d15ed.pth
Loads checkpoint by local backend from path: ../../work_dirs/pointpillars_hv_secfpn_8xb6-160e_kitti-3d-car/epoch_80.pth


/home/javier/sensus-loci/mmdetection3d/mmdet3d/models/dense_heads/anchor3d_head.py:92: UserWarning: dir_offset and dir_limit_offset will be depressed and be incorporated into box coder in the future
  warnings.warn(


In [25]:
# Read pc from bin file
with open(pcd_path, 'rb') as f:
    points = np.fromfile(f, dtype=np.float32, count=-1).reshape([-1, 4])

pcd_bin = o3d.geometry.PointCloud()
pcd_bin.points = o3d.utility.Vector3dVector(points[:, :3])
print(pcd_bin)

PointCloud with 56285 points.


In [26]:
bboxes3d_mmdet = result_mmdet.pred_instances_3d.bboxes_3d.tensor.to('cpu').detach().numpy()
bboxes3d_dair = result_dair.pred_instances_3d.bboxes_3d.tensor.to('cpu').detach().numpy()

# Generate boxes
boxes_mmdet, boxes_dair = generate_cars(10, bboxes3d_mmdet, bboxes3d_dair)


# draw([pcd_bin] + boxes_dair + boxes_mmdet, width=900, height=600, point_size=1)
draw([pcd_bin] + boxes_dair, width=900, height=900, point_size=2)


[ 44.75114  -31.048635  -1.839023]
[58.372177  16.275192  -1.7608439]
[24.033167  -1.0474998 -1.6393907]
[56.51944   25.349005  -1.9438548]
[44.986683  12.826962  -1.7199826]
[30.146275 17.045673 -1.79806 ]
[37.006264  -7.928218  -1.6966988]
[44.477684  17.050266  -1.7670798]
[29.778257  13.562961  -1.7374878]
[52.028023  -4.910778  -1.6688635]
[18.163797   2.2201302 -1.6274009]
[50.649563  -1.6643411 -1.671246 ]
[20.23532   -3.8236191 -1.65694  ]
[48.24469   19.852705  -1.8306892]
[24.072868  -7.769876  -1.6758342]
[38.541016  -4.735648  -1.6788976]
[39.61857   26.672215  -1.9174638]
[Open3D INFO] Window window_8 created.


WebVisualizer(window_uid='window_8')

[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceCandidate
[Open3D INFO] DataChannelObserver::OnStateChange label: ServerDataChannel, state: open, peerid: 0.9105426356436352
[Open3D INFO] DataChannelObserver::OnStateChange label: ClientDataChannel, state: open, peerid: 0.9105426356436352
[Open3D INFO] Sending init frames to window_8.


[341:693][856301] (stun_port.cc:96): Binding request timed out from 163.117.150.x:39300 (enp4s0)
[369:174][856301] (stun_port.cc:96): Binding request timed out from 163.117.150.x:52636 (enp4s0)


In [10]:
def process_label_file(file_path):
    labels = []
    with open(file_path, 'r') as file:
        for line in file:
            label = process_label_line(line)
            labels.append(label)
    return labels

def process_label_line(label_line):
    label_line = label_line.split(' ')
    label = dict()
    label['type'] = label_line[0]
    label['truncated'] = float(label_line[1])
    label['occluded'] = int(label_line[2])
    label['alpha'] = float(label_line[3])
    label['bbox'] = [float(label_line[i]) for i in range(4, 8)]
    label['dimensions'] = [float(label_line[i]) for i in range(8, 11)]
    label['location'] = [float(label_line[i]) for i in range(11, 14)]
    label['rotation_y'] = float(label_line[14])
    label['score'] = float(label_line[15]) if len(label_line) == 16 else None
    return label

def get_lidar_in_cam_T(calib):
    Tr_velo_to_cam = np.array(calib['Tr_velo_to_cam'])
    Tr_velo_to_cam = Tr_velo_to_cam.reshape(3, 4)
    R = Tr_velo_to_cam[0:3, 0:3]
    T = Tr_velo_to_cam[0:3, 3]
    lidar_in_cam_T = np.eye(4)
    lidar_in_cam_T[0:3, 0:3] = R.T
    lidar_in_cam_T[0:3, 3] = -R.T @ T
    return lidar_in_cam_T

def generate_bbox_label(label, calib):
    # Extract label info
    dimensions = label['dimensions']
    position = label['location']
    rotation_y = label['rotation_y']
    
    # Generate bbox
    box = o3d.geometry.TriangleMesh.create_box(width=dimensions[2], height=dimensions[1], depth=dimensions[0])
    box.paint_uniform_color([1.0, 0.0, 0.0]) # Set color to red

    center = box.get_center()

    # Convert position to lidar coordinates
    lidar_in_cam_T = get_lidar_in_cam_T(calib)
    pos_in_cam = np.array(position + [1])

    pos_in_lidar = lidar_in_cam_T @ pos_in_cam

    # Translate box
    box.translate([pos_in_lidar[0], pos_in_lidar[1], pos_in_lidar[2]])

    # Translate box to center
    box.translate([-dimensions[2]/2, -dimensions[1]/2, 0])

    # Rotate box
    center = box.get_center()
    rotation = box.get_rotation_matrix_from_xyz((0, 0, -rotation_y - np.pi/2)) # Notice the negative sign for rotation_y
    box.rotate(rotation, center=center)

    lines = o3d.geometry.LineSet.create_from_triangle_mesh(box)
    # Remove lines that connect non-adjacent points
    lines.lines = o3d.utility.Vector2iVector(np.array([[0, 1], [2, 0], [2, 3], [3, 1], [4, 5], [4, 6], [6, 7], [7, 5], [0, 4], [1, 5], [2, 6], [3, 7]]))
    lines.paint_uniform_color([1, 0, 0])

    return lines

def process_label_file(file_path):
    labels = []
    with open(file_path, 'r') as file:
        for line in file:
            label = process_label_line(line)
            labels.append(label)
    return labels

def process_label_line(label_line):
    label_line = label_line.split(' ')
    label = dict()
    label['type'] = label_line[0]
    label['truncated'] = float(label_line[1])
    label['occluded'] = int(label_line[2])
    label['alpha'] = float(label_line[3])
    label['bbox'] = [float(label_line[i]) for i in range(4, 8)]
    label['dimensions'] = [float(label_line[i]) for i in range(8, 11)]
    label['location'] = [float(label_line[i]) for i in range(11, 14)]
    label['rotation_y'] = float(label_line[14])
    label['score'] = float(label_line[15]) if len(label_line) == 16 else None
    return label

def process_calib_file(file_path):
    calib = dict()
    with open(file_path, 'r') as file:
        for line in file:
            line = line.split(' ')
            if line[0] == 'P2:':
                calib['P2'] = [float(line[i]) for i in range(1, len(line))]
            elif line[0] == 'Tr_velo_to_cam:':
                calib['Tr_velo_to_cam'] = [float(line[i]) for i in range(1, len(line))]
            elif line[0] == 'R0_rect:':
                calib['R0_rect'] = [float(line[i]) for i in range(1, len(line))]
            elif line[0] == 'Tr_imu_to_velo:':
                calib['Tr_imu_to_velo'] = [float(line[i]) for i in range(1, len(line))]
    return calib

In [12]:
register_all_modules()

## Model provided by mmdetection3d
config_mmdet = '../../mmdetection3d/configs/pointpillars/pointpillars_hv_secfpn_8xb6-160e_kitti-3d-car.py'
checkpoint_mmdet = '../../mmdetection3d/checkpoints/hv_pointpillars_secfpn_6x8_160e_kitti-3d-car_20220331_134606-d42d15ed.pth'

## Model trained on DAIR
config_dair = '../configs/dair_pointpillars/pointpillars_hv_secfpn_8xb6-160e_kitti-3d-car.py'
checkpoint_dair = '../../work_dirs/pointpillars_hv_secfpn_8xb6-160e_kitti-3d-car/epoch_80.pth'

## PCD file
pcd_path = 'DAIR/000000/000000.bin'

# ground truth label file
labels_path = 'DAIR/000000/000000_label.txt'
calib_path = 'DAIR/000000/000000_calib.txt'

model_mmdet = init_model(config_mmdet, checkpoint_mmdet, device='cuda:1')
result_mmdet, data_mmdet = inference_detector(model_mmdet, pcd_path)

model_dair = init_model(config_dair, checkpoint_dair, device='cuda:1')
result_dair, data_dair = inference_detector(model_dair, pcd_path)

Loads checkpoint by local backend from path: ../../mmdetection3d/checkpoints/hv_pointpillars_secfpn_6x8_160e_kitti-3d-car_20220331_134606-d42d15ed.pth
Loads checkpoint by local backend from path: ../../work_dirs/pointpillars_hv_secfpn_8xb6-160e_kitti-3d-car/epoch_80.pth


In [20]:
bboxes3d_mmdet = result_mmdet.pred_instances_3d.bboxes_3d.tensor.to('cpu').detach().numpy()
bboxes3d_dair = result_dair.pred_instances_3d.bboxes_3d.tensor.to('cpu').detach().numpy()

# Generate boxes
boxes_mmdet, boxes_dair = generate_cars(11, bboxes3d_mmdet, bboxes3d_dair)

# labels
labels = process_label_file(labels_path)
calib = process_calib_file(calib_path)

label = labels[14]

# Generate bbox
lines = generate_bbox_label(label, calib)

draw([pcd_bin, lines] + boxes_dair, width=900, height=600, point_size=2)


[4.0514374 1.6541232 1.5466162]
[1.8801833 4.5953155 1.5821688]
[3.878345  1.684742  1.5655102]
[1.9794471 4.307521  1.653958 ]
[4.0842986 1.6384016 1.4938534]
[1.9051155 4.0928574 1.591927 ]
[3.8466456 1.6580328 1.5148071]
[1.9339418 4.482034  1.6168844]
[3.6529348 1.6881094 1.6065358]
[1.9963703 4.4733334 1.4839885]
[4.4700203 1.6758642 1.492532 ]
[2.0118613 4.4417753 1.5312076]
[4.5527864 1.6643423 1.5655023]
[1.8662988 4.259191  1.4923718]
[4.4382505 1.7186098 1.7801328]
[2.0015652 4.778095  1.7901412]
[4.2194533 1.6571625 1.5270119]
[1.9509887 4.741187  1.5100716]
[4.203699  1.6529942 1.5301385]
[1.8546014 4.2000256 1.506009 ]
[4.292528  1.7210178 1.491722 ]
[2.0171442 4.4273486 1.4583255]
[Open3D INFO] Window window_10 created.


WebVisualizer(window_uid='window_10')

[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceCandidate
[Open3D INFO] DataChannelObserver::OnStateChange label: ServerDataChannel, state: open, peerid: 0.3203637160731647
[Open3D INFO] DataChannelObserver::OnStateChange label: ClientDataChannel, state: open, peerid: 0.3203637160731647
[Open3D INFO] Sending init frames to window_10.


[212:460][855809] (stun_port.cc:96): Binding request timed out from 163.117.150.x:59712 (enp4s0)
[219:892][855809] (stun_port.cc:96): Binding request timed out from 163.117.150.x:55076 (enp4s0)
[240:407][855809] (stun_port.cc:96): Binding request timed out from 163.117.150.x:42993 (enp4s0)
[250:021][855809] (stun_port.cc:96): Binding request timed out from 163.117.150.x:51065 (enp4s0)
